# 02 - Experimentation - Local

This notebook covers the following steps:

1. Preparing the data using `NVTabular`.
2. Train, and evaluate the `TensorFlow` model.
3. Export a `TensorFlow` model.

## Setup

In [ ]:
%env PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION=python

from google.protobuf.internal import api_implementation
api_implementation._implementation_type = 'python'
print(api_implementation.Type())

In [ ]:
import os
import time
import logging
from datetime import datetime

import cudf
import nvtabular as nvt

import tensorflow as tf
import tensorflow.keras as keras

from src.common import features, utils
from src.data_preprocessing import etl
from src.model_training import trainer

logging.getLogger().setLevel(logging.INFO)
tf.get_logger().setLevel('INFO')

print(f"TensorFlow: {tf.__version__}")

In [ ]:
PROJECT = 'merlin-on-gcp'
REGION = 'us-central1'
BUCKET = 'merlin-on-gcp'
VERTEX_SERVICE_ACCOUNT = f'vertex-sa-mlops@{PROJECT}.iam.gserviceaccount.com'

MOVIES_CSV_DATASET_LOCATION = f"gs://{BUCKET}/movielens25m/dataset/movies.csv"
RATINGS_CSV_DATASET_LOCATION = f"gs://{BUCKET}/movielens25m/dataset/ratings.csv"

WORKSPACE = f"gs://{BUCKET}/movielens25m"
EXPERIMENT_ARTIFACTS_DIR = os.path.join(WORKSPACE, 'experiments')

TENSORBOARD_DISPLAY_NAME = f'tb-{PROJECT}'
EXPERIMENT_NAME = f'{MODEL_DISPLAY_NAME}-experiment'

## Initialize Vertex AI Experiment

In [ ]:
REMOVE_EXPERIMENT_ARTIFACTS = False
if tf.io.gfile.exists(EXPERIMENT_ARTIFACTS_DIR) and REMOVE_EXPERIMENT_ARTIFACTS:
    print("Removing previous experiment artifacts...")
    tf.io.gfile.rmtree(EXPERIMENT_ARTIFACTS_DIR)

if not tf.io.gfile.exists(EXPERIMENT_ARTIFACTS_DIR):
    print("Creating new experiment artifacts directory...")
    tf.io.gfile.mkdir(EXPERIMENT_ARTIFACTS_DIR)

print("Workspace is ready.")

run_id = f"run-local-{datetime.now().strftime('%Y%m%d%H%M%S')}"
EXPERIMENT_RUN_DIR = os.path.join(EXPERIMENT_ARTIFACTS_DIR, EXPERIMENT_NAME, run_id)
print("Experiment run directory:", EXPERIMENT_RUN_DIR)

## 1. Preparing the data using NVTabular

In [ ]:
ETL_OUTPUT_DIR = os.path.join(EXPERIMENT_RUN_DIR, 'etl_output')

In [ ]:
etl.run_etl(
    PROJECT, 
    REGION,  
    MOVIES_CSV_DATASET_LOCATION, 
    RATINGS_CSV_DATASET_LOCATION, 
    ETL_OUTPUT_DIR)

In [ ]:
!gsutil ls {ETL_OUTPUT_DIR}

## 2. Train a TensorFlow model

### Prepare experiment parameters

In [ ]:
LOG_DIR = os.path.join(EXPERIMENT_RUN_DIR, 'logs')
EXPORT_DIR = os.path.join(EXPERIMENT_RUN_DIR, 'model')

In [ ]:
experiment_params = {
    'train_data_file_pattern': os.path.join(ETL_OUTPUT_DIR, 'transformed_data', 'train', '*.parquet'),
    'test_data_file_pattern': os.path.join(ETL_OUTPUT_DIR, 'transformed_data', 'test', '*.parquet'),
    'transform_workflow_dir': os.path.join(ETL_OUTPUT_DIR, 'transform_workflow'),
    'learning_rate': 0.001,
    'batch_size': 1024 * 32,
    'hidden_units': [128, 128],
    'num_epochs': 1
}

vertex_ai.log_params(experiment_params)

### Download the data locally

In [ ]:
if tf.io.gfile.exists('data'):
    tf.io.gfile.rmtree('data')
if tf.io.gfile.exists('transform_workflow'):
    tf.io.gfile.rmtree('transform_workflow')

tf.io.gfile.mkdir('data')
tf.io.gfile.mkdir('data/train')
tf.io.gfile.mkdir('data/test')

In [ ]:
utils.copy_files(experiment_params['train_data_files'], 'data/train')
utils.copy_files(experiment_params['test_data_files'], 'data/test')
utils.download_directory(experiment_params['transform_workflow_dir'], '.')

### Train the model

In [ ]:
recommendation_model = trainer.train(
    train_data_file_pattern='data/train/*.parquet',
    nvt_workflow_dir='transform_workflow',
    hyperparams=experiment_params,
    log_dir=LOG_DIR
)

### Evaluate the model

In [ ]:
eval_loss, eval_mse = trainer.evaluate(
    recommendation_model,
    eval_data_file_pattern='data/test/*.parquet',
    hyperparams=hyperparams
)

vertex_ai.log_metrics({"val_loss": eval_loss, "eval_mae": eval_mae})

### Export the model

In [ ]:
import imp

imp.reload(trainer)

In [ ]:
trainer.export(
    model=recommendation_model,
    nvt_workflow=nvt.Workflow.load('transform_workflow'),
    model_name=MODEL_DISPLAY_NAME,
    export_dir=EXPORT_DIR
)